# ***File dùng để Upsert các Vector đã được lưu ngoài file JSON lên PINECONE***

In [16]:
# Tải các thư viện cần thiết
import json

from pinecone.grpc import PineconeGRPC as Pinecone

In [17]:
# Khởi tạo các biến cần thiết
API_KEY = "pcsk_6Gv7LH_HK8eSqMfipJ6xn6GX33ghLrPJDPvLHkFbgnpyy5tnLWkeMLjt7n7ZPnbbyjisAX"
HOST_DENSE = "https://rag-subjects-dense-y0c06t1.svc.aped-4627-b74a.pinecone.io"
HOST_SPARSE = "https://rag-subjects-sparse-y0c06t1.svc.aped-4627-b74a.pinecone.io"

In [18]:
# Đọc các file JSON
#Tạo hàm đọc
def load_chunks_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    return chunks

In [19]:
# Đọc Dense và Sparse của môn Triết Học
TrietHoc_dense_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Dense.json")
TrietHoc_sparse_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Sparse.json")

# Đọc Dense và Sparse của môn Lịch Sử Đảng
LSD_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Dense.json")
LSD_sparse_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Sparse.json")

In [20]:
pc = Pinecone(api_key=API_KEY)

# Lấy ra Index của Dense Database
dense_index = pc.Index(host=HOST_DENSE)

# Lấy ra Index của Sparse Database
sparse_index = pc.Index(host=HOST_SPARSE)

## ***Upsert môn Triết Học***

In [21]:
dense_index.upsert(
  vectors = TrietHoc_dense_vector,
  namespace="triet-hoc"
)
sparse_index.upsert(
  vectors = TrietHoc_sparse_vector,
  namespace="triet-hoc"
)

upserted_count: 400

## ***Upsert môn Lịch Sử Đảng***

In [22]:
dense_index.upsert(
  vectors=LSD_dense_vector,
  namespace="lich-su-dang"
)
sparse_index.upsert(
  vectors=LSD_sparse_vector,
  namespace="lich-su-dang"
)

upserted_count: 347

## ***Upsert môn Tư Tưởng Hồ Chí Minh***

In [23]:
## Updating......